In [9]:
# %%
import numpy as np
from numpy.fft import rfftn, irfftn, rfftfreq, fftfreq
import matplotlib.pyplot as plt
import vtk
from noises import pink_noise

np.random.seed(42)

# Define the grid dimensions
nx = 16  # Number of points in x direction
ny = 16  # Number of points in y direction
nz = 16  # Number of points in z direction

# Define the range for x, y and z
x_min, x_max = 0, 1  # Range for x
y_min, y_max = 0, 1  # Range for y
z_min, z_max = 0, 1  # Range for z

# Create the vertices
x_vertices = np.linspace(x_min, x_max, nx+1)  # Create x values
y_vertices = np.linspace(y_min, y_max, ny+1)  # Create y values
z_vertices = np.linspace(z_min, z_max, nz+1)  # Create z values

# Use meshgrid to create the 3D vertices grid
XV, YV, ZV = np.meshgrid(x_vertices, y_vertices, z_vertices)

# Create the cell centers
x_centers = (x_vertices[:-1]+x_vertices[1:]) / 2.0  # Create x values
y_centers = (y_vertices[:-1]+y_vertices[1:]) / 2.0  # Create y values
z_centers = (z_vertices[:-1]+z_vertices[1:]) / 2.0  # Create z values

# Use meshgrid to create the 3D grid
X, Y, Z = np.meshgrid(x_centers, y_centers, z_centers)

# Calculate the step sizes in x, y and z directions
dx = x_centers[1] - x_centers[0]
dy = y_centers[1] - y_centers[0]
dz = z_centers[1] - z_centers[0]

# Calculate the wavenumbers in x, y and z directions
kx = 2 * np.pi * fftfreq(nx, dx)
ky = 2 * np.pi * fftfreq(ny, dy)
kz = 2 * np.pi * rfftfreq(nz, dz)

# Use meshgrid to create the 2D grid
KX, KY, KZ = np.meshgrid(kx, ky, kz)
K2 = KX**2 + KY**2 + KZ**2
K = np.sqrt(K2)
INVK2 = np.where(K2==0, 0, 1.0/K2)
KNORM = np.sqrt((KX/np.max(KX))**2 + (KY/np.max(KY))**2 + (KZ/np.max(KZ))**2)
SPEC_FILTER = np.exp(-23.6 * 1024 * (KNORM - 2.0/3.0)**4.)
SPEC_FILTER[KNORM <= 2.0/3.0] = 1

/var/folders/7s/tn4_jx8158vf7rpdxlfyncr40000gn/T/ipykernel_29789/2659457623.py:50: RuntimeWarning: divide by zero encountered in divide
  INVK2 = np.where(K2==0, 0, 1.0/K2)


In [10]:
def create_vtk_image_data(vouts):
    # Create vtkImageData
    imageData = vtk.vtkImageData()
    imageData.SetDimensions(nx+1, ny+1, nz+1)
    imageData.SetSpacing(dx, dy, dz)
    imageData.SetOrigin(x_min, y_min, z_min)
    
    # Create temperature data array
    for key in vouts:
        tempArray = vtk.vtkDoubleArray()
        tempArray.SetName(key)
        tempArray.SetNumberOfComponents(3)
        tempArray.SetNumberOfTuples(nx * ny * nz)
        tempArray.SetArray(vouts[key].ravel(order='F'), nx * ny * nz * 3, 1)
        imageData.GetCellData().SetScalars(tempArray)
    
    return imageData

def write_vtk_image_data(imageData, filename):
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(filename)
    writer.SetInputData(imageData)
    writer.SetDataMode(vtk.vtkXMLWriter.Binary)
    writer.Write()

In [11]:
def curl_3d(F, fft=False):
    if fft:
        return np.array([
            1j*KY*F[2] - 1j*KZ*F[1],
            1j*KZ*F[0] - 1j*KX*F[2],
            1j*KX*F[1] - 1j*KY*F[0]
        ])
    else:
        return np.array([
            irfftn(1j*KY*F[2] - 1j*KZ*F[1]),
            irfftn(1j*KZ*F[0] - 1j*KX*F[2]),
            irfftn(1j*KX*F[1] - 1j*KY*F[0])
        ])

def rfftnv(F):
    # F has a shape of (3, ny, nx, nz)
    return np.array([
            rfftn(F[0]),
            rfftn(F[1]),
            rfftn(F[2])
        ])
def irfftnv(F):
    # F has a shape of (3, ny, nx, nz)
    return np.array([
            irfftn(F[0]),
            irfftn(F[1]),
            irfftn(F[2])
        ])
OMEGAH = np.array([rfftn(pink_noise(ny, nx, nz)),rfftn(pink_noise(ny, nx, nz)),rfftn(pink_noise(ny, nx, nz))])
OMEGAH *= SPEC_FILTER*(ny/100*nx/100*nz/100)
PSIHAT = OMEGAH * INVK2
U = curl_3d(PSIHAT)
E0 = 0.5*np.sum(U**2)

nu = 1e-6
nt = 101
t = 0
CFL = 0.5
dt = CFL/max(np.max(np.abs(U[0]/dx)),np.max(np.abs(U[1]/dy)),np.max(np.abs(U[2]/dz)))
nt_sub = int(np.ceil(100.0/nt/dt))
print(dt, nt_sub)

#imageData = create_vtk_image_data({"U":U.copy().transpose(1,2,3,0).reshape((nx*ny*nz,3))})
#filename = f"results/vort_transport_3d_{0:03d}.vti"
#write_vtk_image_data(imageData, filename)

74.18986885153053 1


In [12]:
def N(OMEGAH):
    OMEGA = irfftnv(OMEGAH)
    PSIHAT = OMEGAH * INVK2
    UH = curl_3d(PSIHAT, fft=True)
    U = irfftnv(UH)
    NONL = np.zeros_like(OMEGAH)
    for i in range(3):
        NONL[i] = rfftn(
              OMEGA[0]*irfftn(1j*KX*UH[i])
            + OMEGA[1]*irfftn(1j*KY*UH[i])
            + OMEGA[2]*irfftn(1j*KZ*UH[i])
            - U[0]*irfftn(1j*KX*OMEGAH[i]) 
            - U[1]*irfftn(1j*KY*OMEGAH[i]) 
            - U[2]*irfftn(1j*KZ*OMEGAH[i])
            ) * SPEC_FILTER
        NONL[i] += nu * (-K2 * OMEGAH[i])
    return NONL

for i in range(nt):
    for _ in range(nt_sub):
        k1 = dt * N(OMEGAH)
        k2 = dt * N(OMEGAH+1.0/2.0*k1)
        k3 = dt * N(OMEGAH+1.0/2.0*k2)
        k4 = dt * N(OMEGAH+k3)
        OMEGAH += 1.0/6.0*(k1 + 2*k2 + 2*k3 + k4)
        t += dt

    PSIHAT = OMEGAH * INVK2
    U = curl_3d(PSIHAT)
    OMEGA = irfftnv(OMEGAH)
    E = 0.5*np.sum(U**2)
    CFL = dt*max(np.max(np.abs(U[0]/dx)),np.max(np.abs(U[1]/dy)),np.max(np.abs(U[2]/dz)))
    print(i, t, CFL, E/E0)

    imageData = create_vtk_image_data({"OMEGA": OMEGA})
    filename = f"results/vort_transport_3d_{i:03d}.vti"
    write_vtk_image_data(imageData, filename)


0 74.18986885153053 0.47983076796198243 0.969751642352778
1 148.37973770306107 0.48472281911512993 0.9559030749466092
2 222.5696065545916 0.4833294381102718 0.9547847182681605
3 296.75947540612214 0.4834354968146575 0.9634361728578769
4 370.9493442576527 0.47971829183864856 0.9794624518935091
5 445.13921310918323 0.4703109791700331 1.0009208678698023
6 519.3290819607138 0.4634970964220805 1.026219427094406
7 593.5189508122443 0.46279028464849004 1.0540209797468962
8 667.7088196637748 0.47310669137586053 1.083163744075914
9 741.8986885153053 0.4911505655086794 1.112615141148725
10 816.0885573668357 0.5203812554592977 1.1414667739359374
11 890.2784262183662 0.5370743514045901 1.1689565874262753
12 964.4682950698967 0.5391264233985287 1.1944845024378938
13 1038.6581639214273 0.5258063133913329 1.217595119533812
14 1112.8480327729578 0.5204771749016837 1.2379393149012907
15 1187.0379016244883 0.5277526015990749 1.2552586245468291
16 1261.2277704760188 0.5232878009002905 1.2694222279189993
